# **Previsão de vendas usando o Keras.**

Esse é um pequeno guia de como criar uma rede neural simples usando o Keras. NÃO irei focar na análise dos dados, apenas no pré-processamento e criação do modelo.

In [1]:
import pandas as pd 

from keras.models import Model
from keras.layers import Input, Dense, Activation

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


### **Carregamento e pré-processamento dos dados**

In [5]:
games = pd.read_csv('/content/Video_Games_Sales_as_at_22_Dec_2016.csv')
games.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


Aqui irei remover as colunas desnecessárias, no caso **Name**, **Year_of_Release**, **Other_Sales**,  **Global_Sales** e **Developer** (porque só preciso das vendas por continente)

In [6]:
games = games.drop(['Name', 'Year_of_Release', 'Other_Sales', 'Global_Sales', 'Developer'], 1)
games.head()

,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,NES,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Wii,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,GB,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN


In [7]:
# Removendo alguns outliers. Não irei remover do Japão pois seus dados já são bem escassos.
games = games.loc[games['NA_Sales'] > 1]
games = games.loc[games['EU_Sales'] > 1]
games.shape

(355, 11)

In [8]:
# Removendo valores nulos
games = games.dropna(axis=0)
games.shape

(258, 11)

##### Separar as *features* e o target.

In [0]:
# As variáveis preditoras são 'Platform', 'Genre', 'Publisher','Critic_Score', 'Critic_Count', 'User_Score', 
# 'User_Count', 'Rating'
X = games.iloc[:, [0, 1, 2, 6, 7, 8, 9, 10]].values

# E as variáveis alvo são 3, uma para cada continente
y_na = games['NA_Sales'].values
y_eu = games['EU_Sales'].values
y_jp = games['JP_Sales'].values

#### *Encode* das *features*.

In [0]:
ohencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(),
                                            [0, 1, 2, 7])], remainder='passthrough')
X = ohencoder.fit_transform(X).toarray()

## **Pré-processamento concluído. Agora, devemos construir a rede neural.**

In [0]:
# Criando a camada de entrada
# 60 é o número de colunas de X
my_input = Input(shape=(60, ))

In [0]:
# Criando a primeira camada oculta
# (camada de entrada + números de saídas) / 2 = (60 + 3) / 2 = 32 aproximadamente
# É necessário dizer com quem a camada oculta se conecta, que nesse caso é com a camada de entrada
my_hidden_layer_1 = Dense(units=32, activation='sigmoid')(my_input)

In [0]:
# Criando a segunda camada oculta
my_hidden_layer_2 = Dense(units=32, activation='sigmoid')(my_hidden_layer_1)

In [0]:
# Criando as camadas de saida, uma para cada continente
# A função de ativação "linear" mantém os valores de saída.
my_output_layer_1 = Dense(units=1, activation='linear')(my_hidden_layer_2)
my_output_layer_2 = Dense(units=1, activation='linear')(my_hidden_layer_2)
my_output_layer_3 = Dense(units=1, activation='linear')(my_hidden_layer_2)

In [0]:
# Criando os modelo com as camadas de entrada e saída.
my_model = Model(inputs=my_input,
                outputs=[my_output_layer_1, my_output_layer_2, my_output_layer_3])

In [0]:
# Compilando o modelo criado
my_model.compile(optimizer='adam', loss='mse')

### **Treinamento do modelo** (processo um pouco demorado, dependendo da capacidade computacional da sua máquina).

In [0]:
my_model.fit(X, [y_na, y_eu, y_jp], epochs=7000, batch_size=100)

Epoch 1/7000
258/258 [==============================] - 0s 1ms/step - loss: 35.5076 - dense_4_loss: 25.5534 - dense_5_loss: 14.9047 - dense_6_loss: 1.2691
Epoch 2/7000
258/258 [==============================] - 0s 35us/step - loss: 34.3781 - dense_4_loss: 19.4315 - dense_5_loss: 12.2031 - dense_6_loss: 1.1219
Epoch 3/7000
258/258 [==============================] - 0s 29us/step - loss: 33.2327 - dense_4_loss: 23.7058 - dense_5_loss: 13.6738 - dense_6_loss: 1.2697
Epoch 4/7000
258/258 [==============================] - 0s 26us/step - loss: 32.1956 - dense_4_loss: 19.1365 - dense_5_loss: 11.1850 - dense_6_loss: 1.0963
Epoch 5/7000
258/258 [==============================] - 0s 33us/step - loss: 31.1817 - dense_4_loss: 17.6806 - dense_5_loss: 10.2748 - dense_6_loss: 1.0976
Epoch 6/7000
258/258 [==============================] - 0s 35us/step - loss: 30.1447 - dense_4_loss: 21.4512 - dense_5_loss: 12.1418 - dense_6_loss: 1.1287
Epoch 7/7000
258/258 [==============================] - 0s 30us/s